In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

model_acronyms = {
    "AutoEncoder": "AE",
    "VariationalAutoEncoder": "VAE",
    "ConditionalVariationalAutoEncoder": "CVAE",
    "ProbabilisticVariationalEncoder": "PVAE",
    "LSTMAutoEncoder": "LSTM-AE"
}
model_order = ["AE", "VAE", "CVAE", "PVAE", "LSTM-AE"]

def plot_metric(df, metric, title, ylabel, filename, output_dir, figsize=(6, 6), ylim=None, unit=""):
    import matplotlib.pyplot as plt
    import seaborn as sns
    import os

    plt.figure(figsize=figsize)
    sns.set_context("notebook", font_scale=1.5)
    df = df[df["Acronym"].isin(model_order)]
    df["Acronym"] = pd.Categorical(df["Acronym"], categories=model_order, ordered=True)

    ax = sns.barplot(
        data=df.sort_values("Acronym"),
        x="Acronym", y=metric,
        hue="Acronym",
        palette="Set2",
        dodge=False,
        legend=False
    )

    if ylim:
        ax.set_ylim(ylim)

    _, y_top = ax.get_ylim()

    for p in ax.patches:
        height = p.get_height()
        if height > y_top * 0.70:
            label_y = height * 0.5
            va = 'center'
            color = 'white'
        else:
            label_y = height
            va = 'bottom'
            color = 'black'

        ax.annotate(f"{height:.2f} {unit}".strip(),
                    (p.get_x() + p.get_width() / 2., label_y),
                    ha='center', va=va, fontsize=22, rotation=90, color=color)

    plt.title(title, fontsize=25)
    plt.ylabel(ylabel, fontsize=22)
    plt.xlabel("", fontsize=22)
    plt.xticks(rotation=45, fontsize=22)
    plt.yticks(fontsize=22)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, filename))
    plt.close()


def plot_performance_metrics(df, output_dir):
    import matplotlib.pyplot as plt
    import seaborn as sns
    import os

    metrics = ["Accuracy", "Precision", "Recall", "F1", "AUC"]
    perf_data = []

    for _, row in df.iterrows():
        for metric in metrics:
            perf_data.append({
                "Model": row["Acronym"],
                "Metric": metric,
                "Score": row[metric]
            })

    perf_df = pd.DataFrame(perf_data)
    perf_df["Model"] = pd.Categorical(perf_df["Model"], categories=model_order, ordered=True)
    perf_df["Metric"] = pd.Categorical(perf_df["Metric"], categories=metrics, ordered=True)

    plt.figure(figsize=(24, 6.00))
    sns.set_context("notebook", font_scale=1.5)

    ax = sns.barplot(data=perf_df, x="Metric", y="Score", hue="Model", palette="Set2")
    plt.ylim(0.6, 1.0)
    plt.title("Model Performance Comparison", fontsize=25)
    plt.ylabel("Score", fontsize=22)
    plt.xlabel("Metric", fontsize=22)

    ax.tick_params(axis='x', labelsize=22)
    ax.tick_params(axis='y', labelsize=22)
    ax.legend(title="Model", fontsize=18, title_fontsize=20, loc='lower left')

    _, y_top = ax.get_ylim()
    for p in ax.patches:
        height = p.get_height()
        if height > y_top * 0.8:
            label_y = height - 0.03
            va = 'top'
            color = 'white'
        else:
            label_y = height
            va = 'bottom'
            color = 'black'

        ax.annotate(f"{height:.2f}",
                    (p.get_x() + p.get_width() / 2., label_y),
                    ha='center', va=va, fontsize=22, rotation=90, color=color)

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "performance_metrics.png"))
    plt.close()



def main(train_csv, test_csv, output_dir="model_comparison_plots"):
    os.makedirs(output_dir, exist_ok=True)

    train_df = pd.read_csv(train_csv)
    test_df = pd.read_csv(test_csv)

    train_df["Acronym"] = train_df["Model"].map(model_acronyms)
    test_df["Acronym"] = test_df["Model"].map(model_acronyms)

    train_grouped = train_df.groupby(["Model", "Acronym"]).mean().reset_index()
    test_grouped = test_df.groupby(["Model", "Acronym"]).mean().reset_index()

    plot_metric(train_grouped, "TrainTime_sec", "Training Time", "Time (sec)", "train_time.png", output_dir, unit="s")
    plot_metric(train_grouped, "TrainPeakRAM_MB", "Training Peak RAM", "RAM (MB)", "train_peak_ram.png", output_dir, unit="MB")
    plot_metric(train_grouped, "CPU_Usage_percent", "Training CPU Usage", "CPU Usage (%)", "train_cpu_usage.png", output_dir, unit="%")
    plot_metric(train_grouped, "DiskSize_KB", "Model Disk Size", "Disk Size (KB)", "disk_size.png", output_dir, unit="KB")
    plot_metric(train_grouped, "Params", "Number of Parameters", "Parameters", "num_parameters.png", output_dir)

    plot_metric(test_grouped, "InferenceTime_ms", "Inference Time", "Time (ms)", "inference_time.png", output_dir, unit="ms")
    plot_metric(test_grouped, "InferencePeakRAM_MB", "Inference Peak RAM", "RAM (MB)", "inference_peak_ram.png", output_dir, unit="MB")
    plot_metric(test_grouped, "CPU_Usage_percent", "Inference CPU Usage", "CPU Usage (%)", "inference_cpu_usage.png", output_dir, unit="%")


    plot_performance_metrics(test_grouped, output_dir)


train_results = os.path.join("Results","Train_runs.csv")
test_results = os.path.join("Results","Test_runs.csv")
output_dir = os.path.join("Results","Plots")

main(train_results, test_results, output_dir)
